# Librerias y dependencias

In [1]:
# Manipulacion de datos
import pandas as pd

# Visualizacion mediante graficas
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import (ColumnDataSource, HoverTool)
from bokeh.transform import factor_cmap
from bokeh.palettes import Viridis256

# inicializacion de Bokeh para entornos Jupyter notebook
output_notebook()

Loading BokehJS ...

# Carga del dataframe

In [2]:
# Load dataset
df = pd.read_csv('./Datasets/Visitas_PIT_procesado.csv')

# Analisis grafico

## Tiempo

In [3]:
# Análisis mensual para 2023 (desde marzo)
df_2023 = df[df['AÑO'] == '2,023']
month_counts_2023 = df_2023['MES'].value_counts().sort_index()
month_data_2023 = month_counts_2023.reset_index()

# Mapear números de mes a nombres
month_names = {1: 'Ene', 2: 'Feb', 3: 'Mar', 4: 'Abr', 5: 'May', 6: 'Jun', 
               7: 'Jul', 8: 'Ago', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dic'}
month_data_2023['Mes_Nombre'] = month_data_2023['MES'].map(month_names)
month_data_2023.columns = ['Mes_Num', 'Cantidad', 'Mes']
source_month_2023 = ColumnDataSource(month_data_2023)

p2_2023 = figure(x_range=month_data_2023['Mes'].tolist(), height=350, 
                title='Distribución Mensual de Visitas - Año 2023 (Mar-Dec)',
                tools='pan,wheel_zoom,box_zoom,reset,save')

p2_2023.vbar(x='Mes', top='Cantidad', width=0.8, source=source_month_2023, color='navy', alpha=0.7)
p2_2023.add_tools(HoverTool(tooltips=[('Mes', '@Mes'), ('Visitas', '@Cantidad')]))
p2_2023.title.text_font_size = "14pt"
show(p2_2023)

# Análisis mensual para 2024 (hasta noviembre)
df_2024 = df[df['AÑO'] == '2,024']
month_counts_2024 = df_2024['MES'].value_counts().sort_index()
month_data_2024 = month_counts_2024.reset_index()

# Mapear números de mes a nombres (usando el mismo diccionario)
month_data_2024['Mes_Nombre'] = month_data_2024['MES'].map(month_names)
month_data_2024.columns = ['Mes_Num', 'Cantidad', 'Mes']
source_month_2024 = ColumnDataSource(month_data_2024)

p2_2024 = figure(x_range=month_data_2024['Mes'].tolist(), height=350, 
                title='Distribución Mensual de Visitas - Año 2024 (Ene-Nov)',
                tools='pan,wheel_zoom,box_zoom,reset,save')

p2_2024.vbar(x='Mes', top='Cantidad', width=0.8, source=source_month_2024, color='darkgreen', alpha=0.7)
p2_2024.add_tools(HoverTool(tooltips=[('Mes', '@Mes'), ('Visitas', '@Cantidad')]))
p2_2024.title.text_font_size = "14pt"
show(p2_2024)

# Resumen comparativo de totales por año (para contexto)
year_counts = df['AÑO'].value_counts().sort_index()
year_data = year_counts.reset_index()
year_data.columns = ['Año', 'Cantidad']
source_year = ColumnDataSource(year_data)

p3 = figure(x_range=[str(x) for x in year_data['Año']], height=350, 
           title='Total de Visitas por Año (Datos Parciales: 2023 Mar-Dec, 2024 Ene-Nov)',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p3.vbar(x='Año', top='Cantidad', width=0.6, source=source_year, color='coral', alpha=0.7)
p3.add_tools(HoverTool(tooltips=[('Año', '@Año'), ('Visitas', '@Cantidad')]))
p3.title.text_font_size = "14pt"
show(p3)

## Ciudades

In [16]:
# Análisis de ciudades de origen (usar la columna correcta del dataset procesado)
origin_counts = df['CIUDAD DE PROCEDENCIA'].value_counts()
origin_data = origin_counts.head(10).reset_index()
origin_data.columns = ['Ciudad', 'Cantidad']
source_origin = ColumnDataSource(origin_data)

p4 = figure(y_range=origin_data['Ciudad'], height=400, 
           title='Top 10 Ciudades de Procedencia (Datos Procesados)',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p4.hbar(y='Ciudad', right='Cantidad', height=0.8, source=source_origin, 
        color='green', alpha=0.7)
p4.add_tools(HoverTool(tooltips=[('Ciudad', '@Ciudad'), ('Visitantes', '@Cantidad')]))
p4.title.text_font_size = "14pt"
show(p4)

# Análisis de ciudades de destino (usando datos procesados)
destination_cols = ['CIUDAD DE DESTINO 1', 'CIUDAD DE DESTINO 2', 
                   'CIUDAD DE DESTINO 3', 'CIUDAD DE DESTINO 4', 
                   'CIUDAD DE DESTINO 5']

# Recopilar todos los destinos, excluyendo NA
all_destinations = []
for col in destination_cols:
    destinations = df[col].dropna()
    destinations = destinations[destinations != 'NA']
    all_destinations.extend(destinations.tolist())

destination_counts = pd.Series(all_destinations).value_counts()
destination_data = destination_counts.head(10).reset_index()
destination_data.columns = ['Ciudad', 'Cantidad']
source_dest = ColumnDataSource(destination_data)

p5 = figure(y_range=destination_data['Ciudad'], height=400, 
           title='Top 10 Ciudades de Destino (Datos Procesados)',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p5.hbar(y='Ciudad', right='Cantidad', height=0.8, source=source_dest, 
        color='orange', alpha=0.7)
p5.add_tools(HoverTool(tooltips=[('Ciudad', '@Ciudad'), ('Visitas', '@Cantidad')]))
p5.title.text_font_size = "14pt"
show(p5)

df['num_destinations'] = 0
for col in destination_cols:
    df['num_destinations'] += ((df[col].notna()) & (df[col] != 'NA')).astype(int)

destination_popularity = df['num_destinations'].value_counts().sort_index()
multi_dest_data = destination_popularity.reset_index()
multi_dest_data.columns = ['Num_Destinos', 'Cantidad']
source_multi = ColumnDataSource(multi_dest_data)

p6 = figure(title="Distribución por Número de Destinos Visitados",
            x_axis_label='Número de Destinos', y_axis_label='Número de Turistas',
            width=600, height=400,
            tools="pan,wheel_zoom,box_zoom,reset,save")

p6.vbar(x='Num_Destinos', top='Cantidad', width=0.8, source=source_multi, 
         color='coral', alpha=0.8)
p6.add_tools(HoverTool(tooltips=[('Destinos', '@Num_Destinos'), 
                                 ('Turistas', '@Cantidad')]))
p6.title.text_font_size = "14pt"
show(p6)

## Análisis Demográfico

In [14]:
# Análisis por género
gender_counts = df['SEXO'].value_counts()
gender_data = gender_counts.reset_index()
gender_data.columns = ['Genero', 'Cantidad']
source_gender = ColumnDataSource(gender_data)

p_gender = figure(x_range=gender_data['Genero'], height=350, 
                 title='Distribución por Género',
                 tools='pan,wheel_zoom,box_zoom,reset,save')

p_gender.vbar(x='Genero', top='Cantidad', width=0.6, source=source_gender, 
              color='skyblue', alpha=0.8)
p_gender.add_tools(HoverTool(tooltips=[('Género', '@Genero'), ('Cantidad', '@Cantidad')]))
p_gender.title.text_font_size = "14pt"
show(p_gender)

# Análisis de países de procedencia
country_counts = df['PAIS DE PROCEDENCIA'].value_counts()
country_data = country_counts.head(10).reset_index()
country_data.columns = ['Pais', 'Cantidad']
source_country = ColumnDataSource(country_data)

p11 = figure(y_range=country_data['Pais'], height=400, 
           title='Top 10 Países de Procedencia',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p11.hbar(y='Pais', right='Cantidad', height=0.8, source=source_country, 
         color='red', alpha=0.7)
p11.add_tools(HoverTool(tooltips=[('País', '@Pais'), ('Visitantes', '@Cantidad')]))
p11.title.text_font_size = "14pt"
show(p11)

## Patrones de Comportamiento de Viaje"

In [15]:
# Análisis de motivos de viaje (usando datos procesados y consolidados)
travel_reason = df['MOTIVO'].value_counts()
reason_data = travel_reason.reset_index()
reason_data.columns = ['Motivo', 'Cantidad']
source_reason = ColumnDataSource(reason_data)

p7 = figure(x_range=reason_data['Motivo'], height=400, 
           title='Distribución de Motivos de Viaje Consolidados (Datos Procesados)',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p7.vbar(x='Motivo', top='Cantidad', width=0.8, source=source_reason,
        color=factor_cmap('Motivo', palette=Viridis256[::30], 
                         factors=reason_data['Motivo'].tolist()))
p7.add_tools(HoverTool(tooltips=[('Motivo', '@Motivo'), ('Cantidad', '@Cantidad')]))
p7.xaxis.major_label_orientation = 1.2
p7.title.text_font_size = "14pt"
show(p7)

# Análisis de medios de transporte (usando datos procesados y consolidados)
from bokeh.palettes import Category20

transport = df['TRANSPORTE'].value_counts()
transport_data = transport.reset_index()
transport_data.columns = ['Transporte', 'Cantidad']
source_transport = ColumnDataSource(transport_data)

# Ensure we don't exceed the number of available colors
num_categories = len(transport_data)
colors = Category20[min(num_categories, len(Category20))]

p8 = figure(x_range=transport_data['Transporte'], height=400, 
           title='Distribución de Medios de Transporte Consolidados (Datos Procesados)',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p8.vbar(x='Transporte', top='Cantidad', width=0.8, source=source_transport, 
        color=factor_cmap('Transporte', palette=colors, 
                         factors=transport_data['Transporte'].tolist()), alpha=0.7)
p8.add_tools(HoverTool(tooltips=[('Transporte', '@Transporte'), ('Cantidad', '@Cantidad')]))
p8.xaxis.major_label_orientation = 1.2
p8.title.text_font_size = "14pt"
show(p8)

# Trip duration analysis
duration_stats = df['TIEMPO DE PERMANENCIA EN DIAS'].describe()
duration_counts = df['TIEMPO DE PERMANENCIA EN DIAS'].value_counts().sort_index()
duration_data = duration_counts.reset_index()
duration_data.columns = ['Dias', 'Cantidad']
source_duration = ColumnDataSource(duration_data)

p9 = figure(height=400, title='Distribución de Duración del Viaje (Días)',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p9.line(x='Dias', y='Cantidad', source=source_duration, line_width=3, color='purple')
p9.circle(x='Dias', y='Cantidad', source=source_duration, size=8, color='purple', alpha=0.7)
p9.add_tools(HoverTool(tooltips=[('Días', '@Dias'), ('Cantidad', '@Cantidad')]))
p9.title.text_font_size = "14pt"
show(p9)

# Group size analysis (how many people travel together)
travelers_stats = df['CANTIDAD DE VIAJEROS'].describe()
travelers_counts = df['CANTIDAD DE VIAJEROS'].value_counts().sort_index()
travelers_data = travelers_counts.reset_index()
travelers_data.columns = ['Tamaño_Grupo', 'Cantidad']
source_travelers = ColumnDataSource(travelers_data)

p10 = figure(height=400, title='Distribución del Tamaño de Grupos de Viaje',
           tools='pan,wheel_zoom,box_zoom,reset,save')

p10.vbar(x='Tamaño_Grupo', top='Cantidad', width=0.8, source=source_travelers, color='teal', alpha=0.7)
p10.add_tools(HoverTool(tooltips=[('Tamaño de Grupo', '@Tamaño_Grupo'), ('Número de Grupos', '@Cantidad')]))
p10.title.text_font_size = "14pt"
show(p10)
